В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

## Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [1]:
from sklearn import datasets, ensemble
import numpy as np
from matplotlib import pyplot as plt

In [2]:
dataset = datasets.load_digits()
x = dataset.data
y = dataset.target

In [15]:
num_elements = x.shape[0]
index = int(0.25*num_elements)

x_train = x[:-index]
y_train = y[:-index]

x_test = x[-index:]
y_test = y[-index:]

In [12]:
x_train.shape, x_test.shape

((1348, 64), (449, 64))

In [13]:
y_train.shape, y_test.shape

((1348, 64), (449, 64))

In [17]:
x = x_test[0,:]

In [33]:
def determine_knn(x, X, y):
    distances = ((X-x)**2).sum(axis=1)
    min_d = distances[0]
    min_t = y[0]
    for d, t in zip(distances, y):
        if d < min_d:
            min_d = d
            min_t = t
    return min_t        

In [34]:
predictions = []
for i in range(x_test.shape[0]):
    x = x_test[i, :]
    predictions.append(determine_knn(x, x_train, y_train))   

In [35]:
predictions = np.array(predictions)

In [36]:
knn_accuracy = float((y_test != predictions).sum())/y_test.shape[0]
knn_accuracy

0.0378619153674833

In [38]:
with open('knn_answer1.txt', 'w') as fout:
    fout.write(str(knn_accuracy))

## Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [26]:
rf_clf = ensemble.RandomForestClassifier(n_estimators = 1000)
rf_clf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=1000)

In [27]:
predictions = rf_clf.predict(x_test)

In [31]:
rf_accuracy = float((y_test != predictions).sum()/y_test.shape[0])
rf_accuracy

0.0645879732739421

In [32]:
with open('knn_answer2.txt', 'w') as fout:
    fout.write(str(rf_accuracy))